# An introduction to profiling in python

Let's take a look at some benchmarking and profiling tools.

## The time module

Starting with doing it yourself using the time module.

`time.perf_counter_ns()` is the maximum resolution time function
returning values in nano-seconds (because floating point seconds
will sometimes not have the resolution to represent short functions).

We'll time a simple MC calculation of pi.


In [1]:
import time
import python_pi

In [2]:
for n in [1000, 10000, 100000, 1000000, 10000000]:
    start = time.perf_counter_ns()
    pi = python_pi.pi_python(n)
    end = time.perf_counter_ns()
    t = end - start
    print(f"{n:10} samples pi is {pi:7f} in {t/1E9:7f} s ({t/n:7f} ns/iteration)")

      1000 samples pi is 3.123123 in 0.000544 s (544.000000 ns/iteration)
     10000 samples pi is 3.139514 in 0.004460 s (445.991700 ns/iteration)
    100000 samples pi is 3.143031 in 0.031771 s (317.710000 ns/iteration)
   1000000 samples pi is 3.140991 in 0.180387 s (180.387292 ns/iteration)
  10000000 samples pi is 3.141678 in 1.763629 s (176.362863 ns/iteration)


## Timeit

We also have `timeit`, a python module designed to give us this kind of information.

It's built into ipython so you can use the 'magic' (%) syntax or you can import it as a module
or run it from the command line.

Remember that
1 s = 1000 ms = 1,000,000 $\mu$s = 1,000,000,000 ns

In [3]:
%timeit python_pi.pi_python(1000)
# Timing stats for the line

172 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
%%timeit

python_pi.pi_python(1000)
# Timing stats for the cell

172 µs ± 38 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
import timeit

t = timeit.Timer('python_pi.pi_python(1000)',setup="from __main__ import python_pi")
tottime = t.timeit(number=10000)
time_per_loop_ms = tottime/10000 * 1.0E6
print(f"1000 samples took {time_per_loop_ms} microseconds")

1000 samples took 172.66654169999995 microseconds


## cProfile

That all tells us how long something takes. Very useful if you are trying to make something faster (are you done yet) but what about finding out where to spend effort? For this we need a proper profiler.

In general profiling comes in two types **deterministic**, where function calls are instrumented and the time spent in each function computed, and **statistical** where the processor reports what it's doing every so often and this is converted into the amount of time spent on each line of code. Being an interpreted language in python we can use a deterministic approach. The same is true of the profiler built into the Matlab environment.

So we'll run our MC code under a profiler. Again we will use the ipython magic but you can do this on the command line too

In [6]:
%%prun -s cumulative
python_pi.pi_python(10000)

         20008 function calls in 0.002 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.002    0.002    0.002    0.002 python_pi.py:4(pi_python)
    20000    0.001    0.000    0.001    0.000 {method 'random' of '_random.Random' objects}
        1    0.000    0.000    0.000    0.000 random.py:126(seed)
        1    0.000    0.000    0.000    0.000 {function Random.seed at 0x1014eeb80}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

You'll need to 
`conda install line_profiler` to do this.

In [ ]:
%load_ext line_profiler
